# Stock Price Prediction with Linear Regression

This notebook demonstrates how to use **yfinance** and **scikit-learn** to predict the next day's closing price of a stock (AAPL) using Linear Regression.

---

In [ ]:
!pip install yfinance scikit-learn pandas numpy matplotlib

In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt


## Download Historical Stock Data

In [ ]:

ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2024-01-01")
data.head()


## Feature Engineering

In [ ]:

data["MA_5"] = data["Close"].rolling(window=5).mean()
data["MA_10"] = data["Close"].rolling(window=10).mean()
data["MA_20"] = data["Close"].rolling(window=20).mean()

data["Next_Close"] = data["Close"].shift(-1)
data.dropna(inplace=True)

data.tail()


## Prepare Features and Target

In [ ]:

features = ["Close", "MA_5", "MA_10", "MA_20", "Volume"]
X = data[features]
y = data["Next_Close"]


## Train-Test Split (Time Series Aware)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


## Train Linear Regression Model

In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)


## Make Predictions

In [ ]:

predictions = model.predict(X_test)


## Evaluate Model

In [ ]:

mae = mean_absolute_error(y_test, predictions)
mae


## Compare Actual vs Predicted

In [ ]:

comparison = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": predictions
}, index=y_test.index)

comparison.head()


## Predict Next Day Closing Price

In [ ]:

latest_data = X.iloc[-1].values.reshape(1, -1)
next_day_prediction = model.predict(latest_data)
next_day_prediction[0]


## Visualization

In [ ]:

plt.figure(figsize=(10,5))
plt.plot(y_test.index, y_test, label="Actual")
plt.plot(y_test.index, predictions, label="Predicted")
plt.legend()
plt.title(f"{ticker} Actual vs Predicted Closing Prices")
plt.show()
